
##  Text embedding (using Large BERT)

In [1]:
!pip install transformers

In [2]:
!pip install sentencepiece

In [3]:
import os
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.functional as F
import pickle
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from transformers import XLNetTokenizer, XLNetModel
import sentencepiece


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT model and tokenizer
# model_name = 'roberta-large'
# tokenizer = RobertaTokenizer.from_pretrained(model_name)
# bert_model = RobertaModel.from_pretrained(model_name)

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
bert_model = XLNetModel.from_pretrained('xlnet-base-cased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state


# Define your binary classification model
class BERT_Embedder(nn.Module):
    def __init__(self, bert_modele,tokenizer):
        super(BERT_Embedder, self).__init__()
        self.bert = bert_modele
        self.tokenizer = tokenizer
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_text):
        inputs = self.tokenizer(input_text, return_tensors="pt").to(device)
        outputs = self.bert(**inputs)
        last_hidden_states = outputs.last_hidden_state[0][0]

        # pooled_output = torch.flatten(self.dropout(last_hidden_states))

        return last_hidden_states#pooled_output


In [5]:
my_embedder = BERT_Embedder(bert_model,tokenizer).to(device)

Load dataset :


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Data/ProcessedData/World/fundemental.csv',index_col=0)
df.head()

,longBusinessSummary,country,industry,sector,forwardPE,beta
OMER,"Omeros Corporation, a clinical-stage biopharma...",United States,Biotechnology,Healthcare,-1.462687,0.957814
CGAU,"Centerra Gold Inc., a gold mining company, eng...",Canada,Gold,Basic Materials,5.580645,1.039580
BFC,Bank First Corporation operates as a holding c...,United States,Banks—Regional,Financial Services,14.056655,0.340248
US.MI,UnipolSai Assicurazioni S.p.A. operates as an ...,Italy,Insurance—Diversified,Financial Services,8.307142,0.637387
001979.SZ,China Merchants Shekou Industrial Zone Holding...,China,Real Estate—Development,Real Estate,11.867925,0.335771


In [10]:
all_texts = df['longBusinessSummary'].to_list()
len(all_texts)

5418

In [11]:
name_IDs = np.load('/content/drive/My Drive/Data/ProcessedData/World/Names/3300_names_id.npy')
stock_names = np.load('/content/drive/My Drive/Data/ProcessedData/World/Names/5400_names.npy')

In [12]:
selected_stock_names = stock_names[name_IDs]
selected_stock_names.shape, stock_names.shape

((3294,), (5418,))

In [13]:
X_embedded = []

In [14]:
X_embedded = []  #  223,1164,1252,1380,1491,1767,1832,2159,2245,2462,2832,2963,2969,3499,3587,3951,3994,4480,4679,4805,4810,5406,5433
i = 0
for index,text in enumerate(all_texts):
    if stock_names[index] in selected_stock_names:
        i += 1
        X_embedded.append(list(torch.detach(torch.Tensor.cpu(my_embedder(text))).numpy()))
        print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
i

3294

In [16]:
np.shape(X_embedded)

(3294, 768)

In [17]:
np.save('/content/drive/My Drive/Data/ProcessedData/World/features/reduced_XLNet',np.array(X_embedded))

---

In [ ]:
dellist = [223,1164,1252,1380,1491,1767,1832,2159,2245,2462,2832,2963,2969,3499,3587,3951,3994,4480,4679,4805,4810,5406,5433]